In [1]:
import requests
import pandas as pd

In [22]:
a = 1
url = "http://127.0.0.1:5000/predict"
data = {'num_reco': 4, 'news_title': 'Tren Regional del Caribe espera el espaldarazo del Gobierno entrante'}

In [23]:
response = requests.post(url, json=data)

In [24]:
print(response.text)

{"resultado":"[{\"news_title\":\"Suspenden a empresas pedreras por robar agua en Nuevo ...\",\"subsec\":\"AUTOMOTORES\"},{\"news_title\":\"CICE inaugura servicio 'Altamira Express' de CMA CGM en ...\",\"subsec\":\"AUTOMOTORES\"},{\"news_title\":\"Por falla en motor, avion de Avianca aterriza de emergencia en Bogota\",\"subsec\":\"AUTOMOTORES\"},{\"news_title\":\"Nace Cevisama Tech, el escaparete mundial de la innovacion ...\",\"subsec\":\"AUTOMOTORES\"}]"}



In [25]:
response.json()['resultado']

'[{"news_title":"Suspenden a empresas pedreras por robar agua en Nuevo ...","subsec":"AUTOMOTORES"},{"news_title":"CICE inaugura servicio \'Altamira Express\' de CMA CGM en ...","subsec":"AUTOMOTORES"},{"news_title":"Por falla en motor, avion de Avianca aterriza de emergencia en Bogota","subsec":"AUTOMOTORES"},{"news_title":"Nace Cevisama Tech, el escaparete mundial de la innovacion ...","subsec":"AUTOMOTORES"}]'

In [26]:
data.keys()

dict_keys(['num_reco', 'news_title'])

In [27]:
pd.read_json(response.json()['resultado'], orient='records')

,news_title,subsec
0,Suspenden a empresas pedreras por robar agua e...,AUTOMOTORES
1,CICE inaugura servicio 'Altamira Express' de C...,AUTOMOTORES
2,"Por falla en motor, avion de Avianca aterriza ...",AUTOMOTORES
3,"Nace Cevisama Tech, el escaparete mundial de l...",AUTOMOTORES


In [14]:
import pandas as pd

In [15]:
pd.DataFrame(data.items(), columns=data.keys())

,num_reco,news_title
0,num_reco,4
1,news_title,Tren Regional del Caribe espera el espaldarazo...


In [9]:
data.items()

dict_items([('parametro1', 1), ('parametro2', 2)])

In [15]:
datos = {'parametro1': [1], 'parametro2': [2]}

In [17]:
pd.DataFrame.from_dict(datos, orient='columns')

,parametro1,parametro2
0,1,2


In [27]:
pd.DataFrame([[1,2]], columns=data.keys()).to_json(orient='records')

'[{"parametro1":1,"parametro2":2}]'

In [10]:
import pandas as pd
from gensim import  models
from gensim.corpora import Dictionary

In [11]:
df_news = pd.read_pickle("./data.pkl")  
clean_txt = [i.split() for i in df_news['news_text_content_clean'].values]

# Creamos la representación de diccionario del documento
dictionary = Dictionary(clean_txt)
corpus = [dictionary.doc2bow(doc) for doc in clean_txt]

model = models.ldamodel.LdaModel.load('model')

topic = [model.get_document_topics(item)[0][0] for item in corpus]
topic_proba = [model.get_document_topics(item)[0][1] for item in corpus]

df_news['topic'] = topic
df_news['topic_proba'] = topic_proba

In [14]:
df_news['news_title'][5]

'Tren Regional del Caribe espera el espaldarazo del Gobierno entrante'

In [2]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import streamlit as st
from gensim import  models
from gensim.corpora import Dictionary
import requests

st.title ("Sistema de Recomendación de Noticias Sobre Clientes Corporativos")
#st.header("Seguimiento de Datos de Entrada")
st.caption("Seguimiento de Datos de Entrada")

#st.markdown("this is the header")
#st.subheader("this is the subheader")
#st.caption("this is the caption")
#st.code("x=2021")
#st.latex(r''' a+a r^1+a r^2+a r^3 ''')

#####################

clientes = pd.read_csv('./clientes.csv')
df_cli =  ((clientes.desc_ciiuu_clase.value_counts()/len(clientes))*100).round(2)
df_cli = df_cli.reset_index().rename({'index':'Actividad Económica', 'desc_ciiuu_clase':'% Noticias'}, axis=1).copy()
df_cli['% Noticias'] = df_cli['% Noticias'].astype(str) + '%'

###################################################
st.header("Actividad Económica Principales")
st.dataframe(df_cli)

#######################################
#Tópicos
#Métricas de Evaluación
#st.header("Tópicos")
#st.header("Métricas de Evaluación")

#######################################

df_news = pd.read_pickle("./data.pkl")  
clean_txt = [i.split() for i in df_news['news_text_content_clean'].values]


# Creamos la representación de diccionario del documento
dictionary = Dictionary(clean_txt)
corpus = [dictionary.doc2bow(doc) for doc in clean_txt]

model = models.ldamodel.LdaModel.load('model')

topic = [model.get_document_topics(item)[0][0] for item in corpus]
topic_proba = [model.get_document_topics(item)[0][1] for item in corpus]

df_news['topic'] = topic
df_news['topic_proba'] = topic_proba

def recomendador_lda(news_title, df, num_reco):
    topic_new = df[df.news_title == news_title].topic.values[0]
    subsec_new = df[df.news_title == news_title].subsec.values[0]
    df = df[df.subsec == subsec_new]
    df = df[df.news_title != news_title].copy()

    return df[df.topic == topic_new].sort_values(by='topic_proba', ascending=False)[['news_title',	'subsec']].reset_index(drop=True)[0:num_reco]

st.header("Sistema de recomendación")
#new = 2

new = st.selectbox(
   "Noticia:",
   df_news.news_title,
   index=None,
   placeholder="Elige la Noticia",
)
if (new != None):
    new_id = df_news[df_news.news_title == new].index[0]

#num_reco = 2

num_reco = st.selectbox(
   "Número de recomendaciones",
   list(range(2, 6)),
   index=None,
   placeholder="Elige el número de recomendaciones",
)

#st.write('You selected:', option)

#reco_id = 0

if (new != None) and (num_reco != None):
    url = "http://127.0.0.1:5000/predict"
    data = {'num_reco': num_reco, 'news_title': df_news.news_title[new_id]}
    response = requests.post(url, json=data)
    df = pd.read_json(response.json()['resultado'], orient='records')
    df = df.rename({'news_title':'Noticia', 'subsec':'Actividad Económica Principal'}, axis=1)
    st.dataframe(df)